In [4]:
import os, json
import numpy as np
import pandas as pd
from sklearn import metrics, preprocessing, cross_validation
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

from nolearn.lasagne import NeuralNet
import lasagne
import theano
import theano.tensor as T

import xgboost as xgb

from ml_metrics import quadratic_weighted_kappa

%matplotlib inline

In [2]:
X = pd.read_csv('onehot_eng_features_train.csv')
Y = pd.read_csv('onehot_eng_features_labels.csv', header=None, index_col=0)
X_test = pd.read_csv('onehot_eng_features_test.csv')
Y = Y[1].as_matrix()

In [16]:
X = X.as_matrix().astype(np.float32)
scalar = preprocessing.StandardScaler()
X = scalar.fit_transform(X)

Y = Y.astype(np.int32)

X_test = X_test.as_matrix().astype(np.float32)

In [21]:
n_classes=8
n_features = X.shape[1]
print(n_features)

977


In [22]:
def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    return quadratic_weighted_kappa(yhat, y)

In [23]:
def nn_model():
    layers = [
        ('input', lasagne.layers.InputLayer),
        ('dense0', lasagne.layers.DenseLayer),
        #('dropout0', lasagne.layers.DropoutLayer),
        #('dense1', lasagne.layers.DenseLayer),
        #('dropout1', lasagne.layers.DropoutLayer),
        #('dense2', lasagne.layers.DenseLayer),
        #('dense3', lasagne.layers.DenseLayer),
        ('output', lasagne.layers.DenseLayer)
    ]

    network = NeuralNet(layers=layers,
                     input_shape=(None, n_features),
                     dense0_num_units=512,
                     #dropout0_p=0.3,
                     #dense1_num_units=128,
                     #dropout1_p=0.1,
                     #dense2_num_units=64,
                     #dense3_num_units=16,
                     output_num_units=n_classes,
                     output_nonlinearity=lasagne.nonlinearities.softmax,
                     update=lasagne.updates.adagrad,
                     update_learning_rate=0.02,
                     #train_split=0.0,
                     #objective_loss_function = binary_accuracy,
                     verbose=1,
                     max_epochs=1)
    return network

epochs = 10

n_folds = 5

# Validation predictions for each fold
X_val_pred = np.zeros((len(X), n_folds))

# auc against the kfold test data
scores = np.zeros((n_folds, epochs))
# auc against the validation data
auc_val_scores = np.zeros((n_folds, epochs))
fold_scores = np.zeros(n_folds)

skf = cross_validation.StratifiedKFold(Y, n_folds=n_folds)

for j, (train_index, test_index) in enumerate(skf):
    print('Fold {} of {}'.format(j+1,n_folds))
    network = nn_model()
    # train
    for i in range(epochs):
        network.fit(X[0:500], Y[0:500])
        # Score on the test data
        pred = network.predict_proba(X.iloc[test_index])[:,1]
        #auc_cv_scores[j,i] = metrics.roc_auc_score(Y[test_index],pred)
        scores[j,i] = eval_wrapper(Y, pred)
        print('score: ', scores[j,i])
        

    # Save the net before starting a new fold
    #network.save_params_to('nn_model/nn05_fold{}.param'.format(str(j).zfill(2)))

Fold 1 of 5
# Neural Network with 504840 learnable parameters

## Layer information

  #  name      size
---  ------  ------
  0  input      977
  1  dense0     512
  2  output       8



ValueError: y_i value out of bounds
Apply node that caused the error: CrossentropySoftmaxArgmax1HotWithBias(Dot22.0, output.b, y_batch)
Toposort index: 11
Inputs types: [TensorType(float64, matrix), TensorType(float64, vector), TensorType(int32, vector)]
Inputs shapes: [(128, 8), (8,), (128,)]
Inputs strides: [(64, 8), (8,), (4,)]
Inputs values: ['not shown', 'not shown', 'not shown']
Outputs clients: [[Sum{acc_dtype=float64}(CrossentropySoftmaxArgmax1HotWithBias.0)], [CrossentropySoftmax1HotWithBiasDx(Elemwise{Inv}[(0, 0)].0, CrossentropySoftmaxArgmax1HotWithBias.1, y_batch)], []]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.